In [26]:
""" 

Run Equilibrium Propagation / Backpropagation on Nanowire Networks for classification. 

Author: Alon Loeffler

Required Files/Folders: learning_functions.py | edamame | asn_nw_01399_nj_06084_seed_1159_avl_10.00_disp_01.00_lx_100.00_ly_100.00.mat

"""

#IMPORTS:

#Append path to Ruomin's Edamame Package (Nanowire Simulations)
import sys
import os

sys.path.append('/import/silo2/aloe8475/Documents/edamame')
saveFig='/import/silo2/aloe8475/Documents/Data/Figures/Equil Prop/'

#import edamame (neuromorphic nanowire python package by Ruomin Zhu)
from edamame import * 
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import loadmat, savemat
import networkx as nx
from tqdm.notebook import tqdm_notebook as tqdm
from IPython.core.debugger import set_trace

import pickle 
import _pickle as cPickle
import gzip

from learning_functions import genGridNW,point_on_line,dist,getWeightedGraph
from learning_functions import calc_cost,setupStimulus,setupSourcesOnly,runTesting,getNWState,calcOutputs

" Create Inputs "
def createSimpleInputs():
    #SIMPLE ROW INPUTS:
    np.random.seed(42)
    numSamples=10
    orig_inputs=np.array((np.array([1,0,1,1,0,1,0]),np.array([0,1,0,1,1,0,1])))
    orig_inputs=orig_inputs.astype('double')
    samples=[[] for i in range(len(orig_inputs))]
    new_inputs=np.array([orig_inputs]*numSamples)

    #Pattern without Noise
    NoNoiseInputs=[]
    for targetClass in range(len(orig_inputs)):
        NoNoiseInputs.append([int(i) for i in orig_inputs[targetClass]])

    #Make Noisy variants of simple patterns:
    for i in range(len(orig_inputs)):
        for j in range(numSamples):
            randVals=np.random.randint(1,3) #choose how many of the 3 sources will have noise
            noiseVals=np.random.randint(3, size=randVals)
            temp=orig_inputs[i][orig_inputs[i]==1].astype('double')
            for val in noiseVals:
                temp[val]=np.double(np.random.random(1))
            new_inputs[j][i][new_inputs[j][i]==1]=temp
    #         samples[i].append(temp)

    #RESHAPE INPUTS INTO 10x3
    row1=[]
    row2=[]
    for i in range(len(new_inputs)):
        row1.append(new_inputs[i][0])
        row2.append(new_inputs[i][1])

    inputs=np.array([row1,row2])
    inputs=inputs.reshape(20,orig_inputs.shape[1])
    targets=np.zeros(20)
    targets[-10:]=1

    #change 0 volts to 0.05
    minVoltage=0.005
    inputs=inputs+minVoltage
    
    return inputs, targets,orig_inputs

" Load Networks "
def buildNetworks(fileName):
    #load data
    nw=loadmat(fileName)
    this_seed=8639
    Network=nw
    connectivity=connectivity__(wires_dict=Network) 

    #fixing file structure from Matlab:
    connectivity.avg_length=connectivity.avg_length[0][0]
    connectivity.number_of_junctions=connectivity.number_of_junctions[0][0]
    connectivity.centroid_dispersion=connectivity.centroid_dispersion[0][0]
    connectivity.dispersion=connectivity.dispersion[0][0]
    # connectivity.generating_number_of_wires=connectivity.generating_number_of_wires[0][0]
    connectivity.gennorm_shape=connectivity.gennorm_shape[0][0]
    connectivity.length_x=connectivity.length_x[0][0]
    connectivity.length_y=connectivity.length_y[0][0]
    connectivity.number_of_wires=connectivity.number_of_wires[0][0]
    connectivity.numOfWires=connectivity.numOfWires[0][0]
    connectivity.numOfJunctions=connectivity.numOfJunctions[0][0]
    connectivity.theta=connectivity.theta[0][0]
    connectivity.this_seed=connectivity.this_seed[0][0]
    
    #find x and y values of each end of each wire 
    xa=connectivity.xa[0]
    xb=connectivity.xb[0]
    ya=connectivity.ya[0]
    yb=connectivity.yb[0]

    #Pick Electrode placement/distance apart:
    
    #DRAINS
    numDrains=3
    ex=np.ones(numDrains)*5#*15
    ey=np.linspace(-1,101,numDrains)#(-1,101,numDrains)

    elecDrain=genGridNW(xa,xb,ya,yb,ex,ey) #generate drain locations in ex, ey coordinates

    #SOURCES
    numSources=9

    #IN A LINE:
    ex=np.ones(numSources)*100#50
    ey=np.linspace(-2,99,numSources)#(-2,99,numSources)
    
    #IN A 3X3 GRID:
    # middleNWx=int(connectivity.length_x/2)+1
    # middleNWy=int(connectivity.length_y/2)-1
    # distBwElecs=10
    # ex=[middleNWx-distBwElecs,middleNWx-distBwElecs,middleNWx-distBwElecs,middleNWx,middleNWx,middleNWx,middleNWx+distBwElecs,middleNWx+distBwElecs,middleNWx+distBwElecs]
    # ey=[middleNWy-distBwElecs,middleNWy,middleNWy+distBwElecs]*3

    elecSource=genGridNW(xa,xb,ya,yb,ex,ey) #generate source locations in ex, ey coordinates


    #remove middle drain and sources for simple patterns:

    elecSource=elecSource[[0,1,2,4,6,7,8]]
    elecDrain=elecDrain[[0,-1]]

    return connectivity,elecSource,elecDrain

#This function changes the write voltage of the drain electrodes. This is where we tune the system so that 
#it prioritises/de-prioritises certain pathways based on targets and non-targets
def change_voltage(stim,sim2,outputVals,target_index,th=0,th2=0,signalType='DC'):
    import copy
    stimulus=copy.deepcopy(stim)
    
    Vi        = np.zeros((int(run_time/dt)))
    maxV      = onAmp*0.75
    n         = len(outputVals)
    cost      = np.zeros(n)
    beta      = 0.1 #change the values here
    grad      = np.zeros((n))
    dsig    = np.array([stimulus[i].signal for i in range(len(stimulus))][:num_drain_training])
    
    #vectorised implementation
    cost=calc_cost(outputVals,target_values) 
    
#     if gradOff == False: #
    grad=beta*(outputVals-target_values) 
#     elif gradOff == True: #if we have hit currents target
#         grad=0
    dsig2=dsig.T+(grad*onAmp)
    for i in range(n): # for each drain:
        if stimulus[i].signal[0] <= maxV and stimulus[i].signal[0] >= -maxV:
            stimulus[i].signal=dsig2.T[i]
        
    print('Costs '+str(cost))
    print('Sum Cost '+str((np.sum(cost))))
    print('Gradients '+str(grad))
    return outputVals,stimulus,cost

#ADD BACKPROP
#This function runs each training epoch and saves the network state at the last timestep of that epoch   
def getNWState_backprop(training_stimulus,state,drains,sources,tmpTargets,run_time=2,dt=0.01,disable_tqdm=False):
    eles = np.append(drains, sources) #all drains
    #     if len(eles) == num_drain_training + num_source_training:
    simVoltage,cost,t = runSim_backprop(connectivity, 
                       stimulus = training_stimulus,
                       junctionMode = 'tunneling',
                       dt = dt, T = run_time, 
                       contactMode = 'preSet',
                       electrodes = eles,
                       findFirst = False,
                       start_state = state,
                       disable_tqdm=disable_tqdm,
                       collapse=True,tmpTargets=tmpTargets,sources=sources,drains=drains)  
    #     JS1 = getJunctionState(training_sim, -1) #save state
    #     else: 
    #         print('Bless you Joel :)')
        
    return simVoltage,cost,t

#Combine drain voltage change with runSim:
def runSim_backprop(Connectivity, 
                    junctionMode='binary', collapse=False,
                    criticalFlux=0.1, maxFlux=1.5e-1,
                    contactMode='farthest', electrodes=None,
                    dt=1e-3, T=10, 
                    stimulus = None,
                    biasType = 'DC',
                    onTime=0, offTime=50000000,
                    onAmp=1, offAmp=0.005,
                    f = 1, customSignal = None,
                    start_state = None,
                    lite_mode = False, save_steps = 1,
                    findFirst = True,
                    disable_tqdm = False,
                    freeze_wire = None, freeze_junction = None, 
                    freeze_TimeStamp = None,tmpTargets=None,
                    sources=[],drains=[]):

    """
    For the case of multi electrodes, stimulus should be in parameters.
    See help(stimulus__) for how to generate stimulus__ objects.
    """

    SimulationOptions = simulationOptions__(dt = dt, T = T,
                                            connectivity = Connectivity, 
                                            contactMode = contactMode,
                                            electrodes = electrodes)

    if ((contactMode == 'preSet') and (len(electrodes) > 2)) or stimulus is not None:
        from sys import exit
        if stimulus == None:
            logging.warning(f'Multiple electrodes detected. Please input stimulus in parameters!')
            exit()
        if len(stimulus) < len(SimulationOptions.electrodes):
            diff = len(SimulationOptions.electrodes) - len(stimulus)
            for _ in range(diff):
                stimulus.append(stimulus__('Drain', T = T, dt = dt))

        if len(stimulus) > len(SimulationOptions.electrodes):
            logging.warning(f'More stimulus than electrodes. Current setup has {len(SimulationOptions.electrodes)} electrodes!')
            exit()
            
        if len(stimulus[0].signal) < len(SimulationOptions.TimeVector):
            logging.warning(f'Stimulus length not correct, current time vector has length {len(SimulationOptions.TimeVector)}!')
            exit()
        SimulationOptions.stimulus = stimulus
            
    elif contactMode == 'boundary':
        SimulationOptions.stimulus[:int(len(SimulationOptions.electrodes)/2)] = [stimulus__(biasType = biasType, 
                                                                                        T = T, dt = dt,
                                                                                        onTime = onTime, offTime = offTime,
                                                                                        onAmp = onAmp, offAmp = offAmp,
                                                                                        f = f, customSignal= customSignal)
                                                                                        for i in range(int(len(SimulationOptions.electrodes)/2))]
    else:
        SimulationOptions.stimulus[0] = stimulus__(biasType = biasType, 
                                                T = T, dt = dt,
                                                onTime = onTime, offTime = offTime,
                                                onAmp = onAmp, offAmp = offAmp,
                                                    f = f, customSignal= customSignal)
    if start_state == None: 
        JunctionState = junctionState__(Connectivity.numOfJunctions, 
                                    mode = junctionMode, collapse = collapse, 
                                    criticalFlux=criticalFlux, maxFlux = maxFlux)
    else:
        from copy import deepcopy
        JunctionState = deepcopy(start_state)
    
    kwdict = dict()
    if (freeze_wire != None) or (freeze_junction != None):
        kwdict = dict(freeze_wire = freeze_wire,
                    freeze_junction = freeze_junction, 
                    freeze_TimeStamp = freeze_TimeStamp)
        
    this_realization,cost = simulateNetwork_backprop(SimulationOptions, Connectivity, JunctionState,tmpTargets, lite_mode, disable_tqdm, save_steps, **kwdict)
    
    if findFirst:
        from edamame.analysis.GraphTheory import findCurrent
        try:
            activation = findCurrent(this_realization, 1)
            logging.info(f'First current path {activation[0][0]} formed at time = {activation[1][0]} s.')
        except:
            logging.info('Unfortunately, no current path is formed in simulation time.')

            
    t,v = calcOutputs(this_realization,sources,drains)

    return this_realization.wireVoltage,cost,t   

def simulateNetwork_backprop(simulationOptions, connectivity, junctionState,tmpTargets, lite_mode = False, disable_tqdm = False, save_steps = 1, **kwargs):
    niterations = simulationOptions.NumOfIterations
    electrodes = simulationOptions.electrodes
    numOfElectrodes = len(electrodes)
    E = connectivity.numOfJunctions
    V = connectivity.numOfWires
    edgeList = connectivity.edge_list
    
    maxV      = onAmp
    n         = num_drain_training #num of drains
    cost      = []
    beta      = 1
    grad      = np.zeros((n)) #set gradients to zero for each epoch  

    Network = network__()
    sampling = np.arange(0, niterations, save_steps)
    if lite_mode:
        Network.connectivity = connectivity__(adjMat = connectivity.adj_matrix)
        Network.filamentState = np.zeros((int(niterations/save_steps), E))
        Network.wireVoltage = np.zeros((int(niterations/save_steps), V))
        Network.electrodeCurrent = np.zeros((int(niterations/save_steps), numOfElectrodes))
        Network.TimeVector = np.zeros(int(niterations/save_steps))
    else:        
        Network.connectivity = connectivity
        Network.TimeVector = simulationOptions.TimeVector
        Network.filamentState = np.zeros((niterations, E))
        Network.junctionVoltage = np.zeros((niterations, E))
        # Network.junctionResistance = np.zeros((niterations, E))
        Network.junctionConductance = np.zeros((niterations, E))
        Network.junctionSwitch = np.zeros((niterations, E), dtype = bool)
        Network.wireVoltage = np.zeros((niterations, V))
        Network.electrodeCurrent = np.zeros((niterations, numOfElectrodes))

    Network.sources = []
    Network.drains = []
    for i in range(numOfElectrodes):
        if np.mean(simulationOptions.stimulus[i].signal) != 0:
            Network.sources.append(electrodes[i])
        else:
            Network.drains.append(electrodes[i])

    if len(Network.drains) == 0:
        Network.drains.append(electrodes[1])

    if 'freeze_wire' in kwargs:
        freeze_wire = kwargs['freeze_wire']
        freeze_TimeStamp = kwargs['freeze_TimeStamp']
    else:
        freeze_TimeStamp = niterations + 1
    
    #Simulation Run:
    new_time=0
    for this_time in tqdm(range(niterations), desc='Running Simulation ', disable = disable_tqdm): #each time step
        
        this_stimulus = np.array([i.signal[this_time] for i in simulationOptions.stimulus])
        
        if this_time == 0:
            print('--------')
            print('Training')
            print('--------\n')
        elif this_time == trainTime:
            print('--------')
            print('Testing')
            print('--------\n')    
        
        if this_time > 0 and this_time < trainTime: #if we are training, change drain voltages, otherwise just use this_stimulus
            this_stimulus[:num_drain_training]=new_stimulus 
                        
        sol = simCore(connectivity, junctionState, this_stimulus, electrodes, simulationOptions.dt)

        if this_time >= freeze_TimeStamp:
            others = np.setdiff1d(range(V), freeze_wire)
            wireVoltage[others] = sol[others]
        else:
            wireVoltage = sol[0:V]
            
        if lite_mode:
            if this_time%save_steps == 0:
                Network.wireVoltage[this_time//save_steps,:] = wireVoltage
                Network.electrodeCurrent[this_time//save_steps,:] = sol[V:]
                Network.filamentState[this_time//save_steps,:] = junctionState.filamentState
                Network.TimeVector[this_time//save_steps] = simulationOptions.TimeVector[this_time]
        else:
            Network.wireVoltage[this_time,:] = wireVoltage
            Network.electrodeCurrent[this_time,:] = sol[V:]
            Network.filamentState[this_time,:] = junctionState.filamentState
            Network.junctionVoltage[this_time,:] = junctionState.voltage
            Network.junctionConductance[this_time,:] = junctionState.conductance
            Network.junctionSwitch[this_time,:] = junctionState.OnOrOff
        
        ## BackProp Implementation
        y = Network.electrodeCurrent[this_time,:num_drain_training]
        
        if tmpTargets[this_time] != -1: #skip rest values
            tarIdx=tmpTargets[this_time]
            y = y/(maxCurrent[tarIdx])  
            
        # current method - run simulation before with just that drain and use current as maxCurrent

            # change to threshold 
            d = allTargets[tarIdx]
            dsig = this_stimulus[:num_drain_training]


            #vectorised implementation
            cost.append(calc_cost(y,d))

            if this_time < trainTime: #if we are training, update beta

                grad=beta*(y-d) #e.g. [y1 vs (d1 = 1), y2 vs (d2 = 0), y3vs (d3 = 0)]
                dsig2=np.array(dsig.T+(grad*onAmp))
                dsig2[tmpTargets[this_time]]=0
                new_stimulus=dsig2

                for q in range(n): # for each drain:
                    if dsig2[q] <= maxV and dsig2[q] >= -maxV:
                        new_stimulus[q]=dsig2[q]
                    elif dsig2[q] < -maxV:
                        new_stimulus[q]=-maxV
                    elif dsig2[q] > maxV:
                        new_stimulus[q]=maxV  

                tmp=range(0,niterations,numDT)
                if this_time in tmp:
                    print('Target ' +str(d))
                    print('Drain V '+str(dsig2))
            else:
                tmp=range(0,niterations,numDT)
                if this_time in tmp:
                    print('Signal ' + str(this_stimulus))
                    print('Target ' +str(d))
                    print('Cost: ' +str(cost[new_time]))
            new_time+=1
            
    Network.numOfWires = V
    Network.numOfJunctions = E
    Network.electrodes = simulationOptions.electrodes
    if len(electrodes) <= 2:
        Network.conductance = Network.electrodeCurrent[:,1]/simulationOptions.stimulus[0].signal[sampling]
    if not lite_mode:
        Network.stimulus = [simulationOptions.stimulus[i] for i in range(numOfElectrodes)]
        Network.junctionResistance = 1/Network.junctionConductance
        
    return Network,cost

def trainingOrder(chosen_seq=1):

    #Parameter 3: Temporal Sequence
    sequences = [1122,2211,1221,2112,1212,2121]
    this_seq  = chosen_seq

    if sequences[this_seq] == 1122:
        order = range(len(inputs))
    elif sequences[this_seq] == 2211:
        order = np.hstack((range(8,12),range(12,16),range(0,4),range(4,8),range(16,20)))
    elif sequences[this_seq] == 1221:
        order = np.hstack((range(0,4),range(8,16),range(4,8),range(16,20)))
    elif sequences[this_seq] == 2112:
        order = np.hstack((range(8,12),range(0,8),range(12,16),range(16,20)))
    elif sequences[this_seq] == 2121:
        order = np.hstack((range(8,12),range(0,4),range(12,16),range(4,8),range(16,20)))
    elif sequences[this_seq] == 1212:
        order = np.hstack((range(0,4),range(8,12),range(4,8),range(12,16),range(16,20)))
    print('Sample Sequence: ' + str(sequences[this_seq]))


    
    return this_seq,order

def associativeLearning(elecSource,elecDrain,allTargets,inputs,connectivity,onAmp,dt):
    #Associative Learning to find max currents for each drain:
    name=r'/import/silo2/aloe8475/Documents/Data/Associative Learning/MaxDrainVals.pkl'
    numClasses=2
    numTestingPerClass=2
    #Parameter 2 = Voltage
    signalType='DC'
    run_time=1
    num_drain_training=1

    if (not os.path.isfile(name)): #if we haven't saved the file

        num_source_training=len(elecSource)
        t=[None]*numClasses
        sim=[None]*numClasses
        currentPaths=[None]*numClasses
        state=[None]*numClasses
        numTrainingSamples=8
        rest=[0]*num_source_training
        restDur=0
        trainOrder=np.array(range(numTrainingSamples))
        np.random.shuffle(trainOrder)

        for classNum in tqdm(range(numClasses)):
            targetClass   = classNum #'z','v','n'
            target_values = allTargets[targetClass]
            chosenDrain   = drain_pool[targetClass]

            targetClassVals=[0,1]
        #     target_values=np.hstack((target_values,0)) #add extra 0 for 4th drain that acts just to balance the network

            trainingInputs  = inputs[targets==targetClassVals[targetClass]][:-numTestingPerClass]
            testingInputs   = inputs[targets==targetClassVals[targetClass]][-numTestingPerClass:]

            #expand each value to run the number of timesteps:
            newRow=np.zeros((len(trainingInputs),int(run_time/dt)+restDur,num_source_training))
            count=0
            for i in trainingInputs[trainOrder]:
                for j in range(int(run_time/dt)+restDur):
                    if j < int(run_time/dt):
                        newRow[count][j]=i
                    else:
                        newRow[count][j]=rest
                count=count+1
            stimulus=np.concatenate(newRow,axis=0).T*onAmp

            newSignal=[]
            for i in range(num_drain_training):
                newSignal.append(stimulus__(biasType='Drain',T=dt*stimulus.shape[1],dt=dt))
            for i in range(len(stimulus)):
                newSignal.append(stimulus__(biasType='Custom',onAmp=onAmp,T=dt*stimulus.shape[1],dt=dt,customSignal=stimulus[i]))
        #     set_trace()
            sim[classNum] = getNWState(newSignal,connectivity,None,[chosenDrain],sources,dt=dt,run_time=len(newSignal[0].signal)*dt)

        maxDrain1=np.max(sim[0][0].electrodeCurrent.T[0])
        maxDrain2=np.max(sim[1][0].electrodeCurrent.T[0])
        
        #save so don't have to repeat
        print('Saving')
        with open(name, 'wb') as f:
            pickle.dump([maxDrain1,maxDrain2], f)   
        print('Saved')
    else:
        print('File Already Exists, Loading:')
        file = open(name,'rb')
        [maxDrain1,maxDrain2] = pickle.load(file)
        print('Loaded')

    return maxDrain1,maxDrain2,numTestingPerClass


def runEquilProp(numClasses,numTrainingSamples,numTestingSamples,trainTime,testTime,num_drain_training,num_source_training,maxCurrent,order,trainingInputs,testingInputs,numDT,restDur,onAmp,onAmpTest,dt,sources,drain_pool):
    print('Voltage Training: ' + str(onAmp)+'V')
    print('Voltage Testing: ' + str(onAmpTest)+'V')
    t                   = [None]*len(order)
    tmpSources          = [[] for i in range(len(order))]
    tmpTargets          = [None]*len(order)
    sim                 = [None]*len(order)
    #Randomise classes:
    i=0
    for val in order: #this method loops through samples

        if val < numTrainingSamples:
            this_sample = trainingInputs[val] #xs
            this_label  = traininglabels[val] #ys
        else:
            ii = val - numTrainingSamples
            this_sample = testingInputs[ii] #xs
            this_label  = testinglabels[ii] #ys

        targetClassVals=[0,1] 

        targetTmp = signal_expand(this_label,numDT) #number of timesteps per target
    #     #array of nans:
        tmpArray=np.empty(restDur)
        tmpArray[:]=-1
        targetTmp = np.append(targetTmp,tmpArray) #add rest #need to remove targets from rest
        tmpTargets[i]=targetTmp 

        for j in range(len(this_sample)): #for each source electrode
            tmp = signal_expand(this_sample[j],numDT)
            tmp = np.append(tmp,np.zeros(restDur)) #add rest
            tmpSources[i].append(tmp)
        i += 1

    #reshape stimulus[i] into 1d:
    tmpSources = np.hstack(np.array(tmpSources))
    tmpTargets = np.hstack(np.array(tmpTargets)).astype(int)

    # Change voltages
    newSources=tmpSources.copy()

    for src in range(len(newSources)):
        a=newSources[src][:trainTime][newSources[src][:trainTime]>0.005]
        b=newSources[src][trainTime:][newSources[src][trainTime:]>0.005]
        newSources[src][:trainTime][newSources[src][:trainTime]>0.005]=onAmp*a
        newSources[src][trainTime:][newSources[src][trainTime:]>0.005]=onAmpTest*b

    tmpDrain=[]
    stimulus=[]
    for j in range(num_drain_training): #create stimulus for each drain electrode
        stimulus.append(stimulus__(biasType='Drain',T=dt*len(newSources[j]),dt=dt))

    for j in range(len(tmpSources)): #create stimulus for each source electrode
        stimulus.append(stimulus__(biasType='Custom',T=dt*len(newSources[j]),dt=dt,customSignal=newSources[j]))

    # set_trace()
    signalLen=len(stimulus[0].signal) 

    #Run simulation
    wireVoltage,cost,t = getNWState_backprop(stimulus,None,drain_pool,sources,tmpTargets,run_time=int(signalLen*dt),dt=dt,disable_tqdm=False)
    
    return t, wireVoltage, trainTime,testTime,cost,tmpTargets,tmpSources
    
"RUN SIMULATION"
print('Loading Networks + Training Data')
inputs,targets,orig_inputs=createSimpleInputs()
allTargets=[[1,0],[0,1]]

fileName='/import/silo2/aloe8475/Documents/Data/Associative Learning/Sparse Networks/asn_nw_01399_nj_06084_seed_1159_avl_10.00_disp_01.00_lx_100.00_ly_100.00'
connectivity,sources,drain_pool=buildNetworks(fileName)
this_seq,order=trainingOrder(chosen_seq=2) #chosen seq = which training order will be chosen

"Choose parameters for training"
allAmpsTraining = [0.5,1,2,5] #Input voltage training
allAmpsTesting  = [0.1,0.2,0.3,0.5] #Input voltage testing
onAmp = allAmpsTraining[2]
onAmpTest = allAmpsTesting[3]

N       = 9
dt      = 0.002 #delta t
numDT   = 300 #number of timesteps per sample
rest    = [0]*N 
restDur = 0 #number of rest timesteps between samples

"Run Simulation to calculate approximate target currents"
print('Running Associative Learning Preprocessing')
maxDrain1,maxDrain2,numTestingPerClass=associativeLearning(sources,drain_pool,allTargets,inputs,connectivity,onAmp,dt)

"Split into training and testing"
targetClassVals=[0,1]
trainingInputs  = np.array([inputs[targets==targetClassVals[0]][:-numTestingPerClass],inputs[targets==targetClassVals[1]][:-numTestingPerClass]]).reshape(-1,orig_inputs.shape[1])
testingInputs   = np.array([inputs[targets==targetClassVals[0]][-numTestingPerClass:],inputs[targets==targetClassVals[1]][-numTestingPerClass:]]).reshape(-1,orig_inputs.shape[1])

#Define Training and Testing labels
traininglabels=np.zeros(len(trainingInputs))
traininglabels[:int(len(trainingInputs)/2)]=0
traininglabels[int(len(trainingInputs)/2):int(len(trainingInputs)/2)*2]=1
# traininglabels[int(len(trainingInputs)/3)*2:]=2
traininglabels=traininglabels.astype(int)

testinglabels=np.zeros(len(testingInputs))
testinglabels[:int(len(testingInputs)/2)]=0
testinglabels[int(len(testingInputs)/2):int(len(testingInputs)/2)*2]=1
# testinglabels[int(len(testingInputs)/3)*2:]=2
testinglabels=testinglabels.astype(int)

"Run EquilProp Sim"
print('Running Equil Prop')
num_drain_training  = 2
num_source_training = orig_inputs.shape[1]
maxCurrent          = [maxDrain1,maxDrain2]#,maxDrain3]
numClasses          = 2#3

numTrainingSamples  = len(trainingInputs)
numTestingSamples   = len(testingInputs)
trainTime  = numTrainingSamples*(numDT+restDur)
testTime   = numTestingSamples*(numDT+restDur)

t,wireVoltage,trainTime,testTime,cost,tmpTargets,tmpSources = runEquilProp(numClasses,numTrainingSamples,numTestingSamples,trainTime,testTime,num_drain_training,num_source_training,maxCurrent,order,trainingInputs,testingInputs,numDT,restDur,onAmp,onAmpTest,dt,sources,drain_pool)

"Calc test accuracy"
test_acc=np.sum(np.argmax([t[0],t[1]],axis=0)[trainTime:]==tmpTargets[trainTime:])/len(tmpTargets[trainTime:])

Loading Networks + Training Data
Sample Sequence: 1221
Running Associative Learning Preprocessing
Running Equil Prop
Voltage Training: 2V
Voltage Testing: 0.5V


--------
Training
--------

Target [1, 0]
Drain V [0.         0.00143707]
Target [1, 0]
Drain V [0.         0.37704416]
Target [1, 0]
Drain V [0.         0.62110019]
Target [1, 0]
Drain V [0.        0.7003906]
Target [0, 1]
Drain V [0.49564707 0.        ]
Target [0, 1]
Drain V [1.07871294 0.        ]
Target [0, 1]
Drain V [1.05772872 0.        ]
Target [0, 1]
Drain V [0.99556792 0.        ]
Target [0, 1]
Drain V [0.78583148 0.        ]
Target [0, 1]
Drain V [0.84940661 0.        ]
Target [0, 1]
Drain V [0.6974635 0.       ]
Target [0, 1]
Drain V [0.84595249 0.        ]
Target [1, 0]
Drain V [0.         1.70663649]
Target [1, 0]
Drain V [0.         1.03393764]
Target [1, 0]
Drain V [0.        0.4785487]
Target [1, 0]
Drain V [0.         0.30093939]
--------
Testing
--------

Signal [0.         0.         0.30627243 0.005      0.5025     0.02572521
 0.005      0.5025     0.005     ]
Target [1, 0]
Cost: [0.40239907 0.00495034]
Signal [0.        0.        0.0350258 0.005     0.5025    0.50

In [27]:
test_acc

0.9033333333333333